In [28]:
# 필요한 라이브러리 설치
# !pip install transformers pandas torch gradio requests beautifulsoup4

from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline
from bs4 import BeautifulSoup
import requests
import gradio as gr

In [29]:
# 1. 기사 크롤링 함수
def extract_article_content(url):
    """
    주어진 URL에서 기사 본문을 추출.
    """
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 사이트별 본문 CSS 셀렉터
        selectors = [
            "div#dic_area",  # 네이버 뉴스
            "div.article_body",  # 예: 특정 뉴스 사이트
            "div#articleBodyContents",  # 예: 다음 뉴스
            "article",  # 일반적인 <article> 태그
            "div.content",  # 기타 사이트
        ]

        # 본문 추출
        for selector in selectors:
            content = soup.select_one(selector)
            if content and content.get_text(strip=True):
                return content.get_text(separator=" ", strip=True)
        
        # 본문을 찾지 못한 경우
        return "기사를 읽을 수 없습니다. 링크를 확인하세요."
    
    except Exception as e:
        return f"오류 발생: {e}"

In [30]:
# 2. 모델 초기화 함수
def initialize_models():
    """
    RoBERTa 분류기와 T5 요약 모델 초기화.
    """
    # 로버타 모델 초기화
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
    model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

    # 요약 모델 초기화
    summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")
    
    return classifier, summarizer

In [31]:
# 3. 기사 분석 함수
def analyze_article(url):
    """
    URL에서 기사를 크롤링하고 요약 및 분류.
    """
    # 기사 본문 추출
    article_content = extract_article_content(url)
    if "오류 발생" in article_content or "기사를 읽을 수 없습니다" in article_content:
        return article_content, "분류 불가", "요약 불가"
    
    # 모델 초기화
    classifier, summarizer = initialize_models()

    # 가짜 뉴스 판별
    try:
        classification_result = classifier(article_content[:512])[0]["label"]  # 'LABEL_0' 또는 'LABEL_1'
        classification = "진짜 뉴스" if classification_result == "LABEL_0" else "가짜 뉴스"
    except Exception as e:
        classification = f"분류 실패: {e}"

    # 기사 요약
    try:
        summary_result = summarizer(article_content[:512], max_length=50, min_length=10, do_sample=False)
        summary = summary_result[0]["summary_text"]
    except Exception as e:
        summary = f"요약 실패: {e}"

    return article_content, classification, summary

In [32]:
# 4. Gradio 인터페이스 정의
def gradio_interface(url):
    article_content, classification, summary = analyze_article(url)
    return article_content, classification, summary

# Gradio 설정
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="기사 링크", placeholder="뉴스 링크를 입력하세요"),
    outputs=[
        gr.Textbox(label="기사 본문"),
        gr.Textbox(label="가짜 뉴스 여부"),
        gr.Textbox(label="요약된 내용")
    ],
    title="가짜 뉴스 탐지 및 요약기",
    description="기사 링크를 입력하면 본문을 크롤링하고, 요약 및 가짜 뉴스 여부를 판단합니다."
)


In [33]:
# 실행
if __name__ == "__main__":
    interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://dcd8d2822c0cc29cae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 512). Running this sequence through the model will result in indexing errors
